In [123]:
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon
import folium
import pandas as pd

In [124]:
df = pd.read_csv('./data/opendata_project.csv')
df

,row_number,project_id,name_en,name_th,propertytype_id,propertytype_name_en,propertytype_name_th,price_min,developer_id,developer_name_en,...,facility_park,facility_playground,facility_pool,facility_security,date_created,date_finish,date_updated,source,url_project,day
0,9665.0,project_15493,Silanta 8 Life,สิลันตา 8 ไลฟ์,3,Townhome,ทาวน์โฮม,1590000.0,developer_7089,"Prevision Realestate Co.,Ltd.",...,1.0,NaN,NaN,1.0,2019-05-24 11:10:00,2020-01-01 07:00:00,2022-06-22 11:30:03,baania,https://baania.com/th/project/สิลันตา-8-ไลฟ์-p...,2023-08-31
1,22645.0,project_16938,Baanbaan Vibhavadi 20,บ้านบ้าน วิภาวดี 20,1,Detached House,บ้าน,26000000.0,developer_10504,"RISING PLUS DEVELOPMENT CO., LTD.",...,NaN,NaN,NaN,NaN,2019-11-05 09:51:19,NaN,2023-01-05 14:30:03,baania,https://baania.com/th/project/บ้านบ้าน-วิภาวดี...,2023-08-31
2,23448.0,project_11503,The Metro Rama 9 - Krungthep Kreetha,เดอะ เมทโทร พระราม 9 - กรุงเทพกรีฑา,3,Townhome,ทาวน์โฮม,3790000.0,developer_1751,"PROPERTY PERFECT PUB CO., LTD.",...,NaN,NaN,1.0,NaN,2017-11-08 09:46:33,2018-07-31 07:00:00,2023-08-03 13:00:04,baania,https://baania.com/th/project/เดอะ-เมทโทร-พระร...,2023-08-31
3,11342.0,project_12358,The Plant Citi Ladprao 71,เดอะ แพลนท์ ซิตี้ ลาดพร้าว 71,3,Townhome,ทาวน์โฮม,2990000.0,developer_1026,"Preuksa Real Estate Co.,Ltd.",...,1.0,1.0,NaN,1.0,2017-12-07 11:02:53,NaN,2020-12-04 10:42:16,baania,https://baania.com/th/project/เดอะ-แพลนท์-ซิตี...,2023-08-31
4,10009.0,project_2871,Baan PS Home 2,บ้านพีเอสโฮม 2,1,Detached House,บ้าน,3990000.0,developer_10510,NaN,...,NaN,NaN,NaN,NaN,2016-12-01 10:48:22,NaN,2020-12-04 10:39:17,baania,https://baania.com/th/project/บ้านพีเอสโฮม-2-p...,2023-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24266,9966.0,project_2454,The Bangkok Thonglor,เดอะ แบงค็อค ทองหล่อ,2,Condo,คอนโด,19000000.0,developer_114,"Land And Houses Public Co.,Ltd.",...,1.0,1.0,1.0,1.0,2016-11-11 10:54:02,2019-11-30 07:00:00,2021-06-09 14:00:08,baania,https://baania.com/th/project/เดอะ-แบงค็อค-ทอง...,2023-08-31
24267,9421.0,63c7a4c37866db00121113ca,Baan Rangsiya Udomsuk 58,บ้านรังสิยา อุดมสุข 58,1,Detached House,บ้าน,0.0,developer_10510,NaN,...,NaN,NaN,NaN,1.0,2023-01-18 14:50:27,NaN,2023-01-18 15:30:02,baania,https://baania.com/th/project/บ้านรังสิยา-อุดม...,2023-08-31
24268,16103.0,61891ef429f24100138ad19b,Lalin Town Lanceo Crib 3 Bangna-Suvarnabhumi,ลลิลทาวน์ แลนซีโอ คริป 3 บางนา-สุวรรณภูมิ,20000,Twin house,บ้านแฝด,3000000.0,developer_4982,"LALIN PROPERTY PUB CO., LTD.",...,1.0,NaN,NaN,1.0,2021-11-08 19:58:28,NaN,2023-02-15 14:00:05,baania,https://baania.com/th/project/ลลิลทาวน์-แลนซีโ...,2023-08-31
24269,19703.0,project_1313,MUBAN MUKDA VILLE,บ้านมุกดาวิลล์,1,Detached House,บ้าน,2950000.0,developer_1191,"Indy Land and Houses Group Co., Ltd.",...,1.0,NaN,NaN,1.0,2016-04-22 15:15:07,NaN,2021-11-04 17:30:08,baania,https://baania.com/th/project/บ้านมุกดาวิลล์-p...,2023-08-31


In [125]:
df = df[['project_id', 'name_th', 'price_min', 'developer_name_th', 'latitude', 'longitude', 'url_project', 'propertytype_name_th','date_created']]

In [126]:
# df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
# df = df[df['date_created'] > pd.to_datetime('2020-01-01')]

In [127]:
df['price_min'] = pd.to_numeric(df['price_min'], errors='coerce')
df['date_created'] = pd.to_datetime(df['date_created'], format='%Y-%m-%d', errors='coerce')
df = df[(df['price_min'] < 7000000) & (df['price_min'] > 3000000)]
# df = df[df['date_created'] > '2023-01-01']

/var/folders/cj/7jh5n4rs0_1c0m8b3059wx000000gn/T/ipykernel_17416/3561587924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_min'] = pd.to_numeric(df['price_min'], errors='coerce')
/var/folders/cj/7jh5n4rs0_1c0m8b3059wx000000gn/T/ipykernel_17416/3561587924.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_created'] = pd.to_datetime(df['date_created'], format='%Y-%m-%d', errors='coerce')


In [128]:
df = df[(df['propertytype_name_th'] == 'บ้าน') | (df['propertytype_name_th'] == 'บ้านแฝด')]

In [129]:
df.dropna(subset=['developer_name_th'], inplace=True)

In [130]:
center_point = (13.755052813872567, 100.47036465177612)
radius_km = 20

In [131]:
def is_within_radius(center, point, radius):
    if pd.isnull(point[0]) or pd.isnull(point[1]):
        return False
    return geodesic(center, point).km <= radius

# Apply the function to filter records
df['is_within_radius'] = df.apply(lambda row: is_within_radius(center_point, (row['latitude'], row['longitude']), radius_km), axis=1)
filtered_df = df[df['is_within_radius']]

In [132]:
# Create a map centered around the center point
map_center = folium.Map(location=center_point, zoom_start=13)

# Add the center point to the map
folium.Marker(center_point, popup='Center Point', icon=folium.Icon(color='red')).add_to(map_center)

# Add the filtered points to the map
for _, row in filtered_df.iterrows():
    popup_content = f"Field 1: {row['name_th']}<br>Field 2: {row['developer_name_th']}<br>Field 3: {row['price_min']}<br>Field 4: {row['url_project']}"  # Modify this line to include the specific fields you want to display
    folium.CircleMarker(location=(row['latitude'], row['longitude']), radius=2, color='blue', fill=True, fill_opacity=0.6, popup=popup_content).add_to(map_center)

# Display the map
map_center